### 지도에 부동산 정보 나타내기

부동산과 지도는 뗼수 없는 관계이다.
부동산은 숫자만 보고 이해하기 힘들기에 부동산 가격이라는것이 입지와 밀접한 관련이 있는 만큼 지역의 입지 , 즉 자리를 확인하지 않고 온전히 이해하기 힘들다.
부동산 데이터를 지도와 함께 살펴보자.
1. folium을 이용해 지도 만들기
- 지도위에 데이터를 시각화 할때 folium을 많이 사용한다.

In [4]:
import folium
# 서울로 좌표찍어서 생성
seoul_map = folium.Map(location=[37.5614378,126.9751701])
seoul_map

In [5]:
# 맵 확대해서 생성
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=14)
seoul_map

In [7]:
# 특정위치 표시
folium.Marker([37.5614378,126.9751701], popup='서울시청').add_to(seoul_map)
seoul_map

In [8]:
import pandas as pd

school_path = r'/Users/parkjunhyeok/Desktop/부동산데이터분석/2018년_공시대상학교정보(전체).xlsx'
school_raw = pd.read_excel(school_path)
school_data = school_raw[['정보공시 \n 학교코드', '학교명', '위도', '경도' ]]
school_data.columns = ['학교코드', '학교명','위도', '경도']
school_df = school_data.drop_duplicates()

/opt/homebrew/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
school_df.head()

,학교코드,학교명,위도,경도
0,S000003511,서울교육대학교부설초등학교,37.490739,127.015424
1,S000003563,서울대학교사범대학부설초등학교,37.577017,127.003091
2,S010000737,서울개원초등학교,37.481618,127.058738
3,S010000738,서울개일초등학교,37.486214,127.057742
4,S010000741,서울구룡초등학교,37.481193,127.051755


In [10]:
def graduate_preprocrssing(path):
    raw_graduate = pd.read_excel(path, sheet_name='2018_졸업생의 진로 현황(중)')
    select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
    graduate_data = select_col.drop(0)
    graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
    graduate_data['외고'] =  pd.to_numeric(graduate_data['외고']) 
    graduate_data['졸업자'] =  pd.to_numeric(graduate_data['졸업자']) 
    graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
    
    def get_sido(x):
        temp = x.split(' ')[0]
        if len(temp) != 4:
            return temp[:2]
        else:
            return temp[0] + temp[2]
    
    graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_sido)
    graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])
    
    graduate_data.at[588,'시도'] = '부산'
    graduate_data.at[588,'구군'] = '기장군'
    graduate_data.at[3011,'시도'] = '경북'
    graduate_data.at[3011,'구군'] = '예천군'
    
    return graduate_data

graduate_path = r'/Users/parkjunhyeok/Desktop/부동산데이터분석/2018년 2차_졸업생의 진로 현황(전체).xlsx'
gradu_df = graduate_preprocrssing(graduate_path)

/opt/homebrew/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
total_school_df = pd.merge(gradu_df, school_df, how='inner', right_on='학교코드', left_on='학교코드')

In [12]:
seoul_school = total_school_df[total_school_df['시도'] == '서울']
good_school = seoul_school[seoul_school['총합'] > 0]
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=11)

for n in good_school.index:
    folium.Marker([ good_school['위도'][n], good_school['경도'][n] ], popup=good_school['학교명_x'][n]).add_to(seoul_map)

In [13]:
seoul_map


In [ ]:
seoul_school['비율'] = seoul_school['총합'] /seoul_school['졸업자'] * 100
good_school = seoul_school[seoul_school['비율'] >= 3]
bad_school = seoul_school[seoul_school['비율'] < 3]
seoul_map = folium.Map(location=[37.5614378,126.9751701], zoom_start=11)

for n in good_school.index:
    folium.CircleMarker([ good_school['위도'][n], good_school['경도'][n] ], color='crimson',fill_color='crimson', radius=7).add_to(seoul_map)

for n in bad_school.index:
    folium.CircleMarker([ bad_school['위도'][n], bad_school['경도'][n] ], color='#3186cc',fill_color='#3186cc',  radius=7).add_to(seoul_map)